In [ ]:
from sklearn import metrics
# from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler

from collections import defaultdict
from collections import Counter

from time import time

import pandas as pd
import numpy as np

import pprint as pp

import src.utilities as ut

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
%matplotlib inline
sns.set()


# Import data set:

In [ ]:
pd.options.mode.chained_assignment = None


df_full = ut.import_dataset(encoding='utf-8')
df_full.head()

,text,class
0,"""Ξεκίνησε το πρωί ενώπιον των δικαστών του Μικ...",Ανθρωποκτονία
1,"""Στο άλλο άκρο έχουν περάσει πλέον οι διαδηλώσ...",Ανθρωποκτονία
2,"""Δεν λέει κουβέντα ο Γερμανός καταδικασμένος π...",Ανθρωποκτονία
3,"""2,5 χρόνια μετά από τον θάνατο της Ειρήνης Λα...",Ανθρωποκτονία
4,"""Μία εβδομάδα μετά την αρχική του εκτίμηση ότι...",Ανθρωποκτονία


Kept only the column of interest leaving out the column with `class` labels.

In [ ]:
df = df_full[['text']]
df['text'] = df['text'].astype('string')

df.head()

,text
0,"""Ξεκίνησε το πρωί ενώπιον των δικαστών του Μικ..."
1,"""Στο άλλο άκρο έχουν περάσει πλέον οι διαδηλώσ..."
2,"""Δεν λέει κουβέντα ο Γερμανός καταδικασμένος π..."
3,"""2,5 χρόνια μετά από τον θάνατο της Ειρήνης Λα..."
4,"""Μία εβδομάδα μετά την αρχική του εκτίμηση ότι..."


# Text processing:

Now we will create a **pipeline** in order to process the textual data we have. The **pipeline** will consist of the following steps:

- Lowercase all characters
- Drop numbers
- Remove punctuation
- Remove stopwords. The list of stopwords can be found here.
- Remove intonation from words.
- Tokenization
- Lemmatization. The process during which all words are turned into their roots.
- Finally, single letter words are dropped, too.

Of course, the above **pipeline** can be adjusted by moving or adding other steps depending on the use case.

In [ ]:
pipeline = [str.lower, ut.drop_numbers, ut.remove_punctuation, ut.remove_stopwords, ut.remove_intonation, 
    ut.tokenize_regex, ut.lemmatize, ut.drop_single_letter_words, ut.remove_final_s]

Apply the process pipeline:

In [ ]:
df['tokens'] = df['text'].apply(ut.process, pipeline=pipeline)

Add column for the number of tokens:

In [ ]:
df['num_tokens'] = df['tokens'].map(len)

See `datatypes` of resulting columns:
We want to make sure we are dealing with `strings`.

In [ ]:
datatypes = df.dtypes
print(datatypes)

text           string
length          Int64
log_length    Float64
tokens         object
num_tokens      int64
dtype: object


Turn **tokens** column from pandas `object` to `string` and see data types again:

In [ ]:
df['tokens'] = df['tokens'].astype('string')

datatypes = df.dtypes
print(datatypes)

text           string
length          Int64
log_length    Float64
tokens         string
num_tokens      int64
dtype: object


The last thing in order to be able to proceed to classification analysis. We need to add the **labels** column back. Therefore, we set up the dataset with both the processed text & their respective classes.

In [ ]:
df['label'] = df_full['class']
df.columns = ['text', 'length', 'log_length','tokens', 'num_tokens', 'label']

Check out the resulting `DataFrame`.

In [ ]:
df.head()

,text,length,log_length,tokens,num_tokens,label
0,"""Ξεκίνησε το πρωί ενώπιον των δικαστών του Μικ...",1627,7.394493,ξεκινησε πρωι ενωπιον δικαστων μικτου ορκωτο ε...,1190,Ανθρωποκτονία
1,"""Στο άλλο άκρο έχουν περάσει πλέον οι διαδηλώσ...",3464,8.150179,αλλο ακρο εχω περαω πλεον διαδηλωσει σε ο ΗΠΑ ...,2589,Ανθρωποκτονία
2,"""Δεν λέει κουβέντα ο Γερμανός καταδικασμένος π...",2914,7.977282,λεει κουβεντα γερμανο καταδικασμενο παιδοφιλο ...,2140,Ανθρωποκτονία
3,"""2,5 χρόνια μετά από τον θάνατο της Ειρήνης Λα...",2702,7.901748,χρονο απο θανατο ειρηνη λαγουδη μητερα τριο πα...,2086,Ανθρωποκτονία
4,"""Μία εβδομάδα μετά την αρχική του εκτίμηση ότι...",2371,7.771067,ένα εβδομαδα αρχικο εκτιμηση οτι μαντλιν μακκα...,1808,Ανθρωποκτονία


# Feature Selection:

We saw that our two models have a significant difference as far as their accuracy is concerned.

- The `Naïve Bayes` classifier produces results with `0.84%` accuracy.
- The `LinearSVG` classifier produces results with `0.91%` accuracy.

Following, we are going to implement some **feature selection** techniques in order to examine whether we can make *better* predictions after having reduced the *feature space* of our data.